# Введение в NumPy
NumPy - фундаментальный модуль для научных вычислений. Он включает в себя высокоэффективную реализацию многомерных массивов, методов линейной алгебры, генераторов случайных чисел и т.д. Многие научные библиотеки, включая библиотек для машинного обучения, используют NumPy.

Все данные, которые подаются на вход, обрабатываются и получаются на выходе методов анализа данных и машинного обучения, представлены в виде многомерных массивов. Методы обработки этих данных реализованы как операции над этими массивами. Поэтому эффективное хранение и обработка таких массивов данных является абсолютно фундаменательной задачей для высокоэффективных вычислений.

В этом уроке мы рассмотрим как NumPy добивается такой эффективности, какие операции поддерживает и многое другое.

Начнем с импорта NumPy и проверки версии

In [1]:
import numpy
numpy.__version__

'1.13.3'

Обычно для удобства при импорте NumPy указывается алиас np

In [2]:
import numpy as np

Как обычно вы можете получить документацию по модулю с помощю знака `?` после названия

In [3]:
np?

### Реализация типов в Python
Давайте сначала рассмотрим пример одного и того же кода на C и Python
```C
/* C code */
int result = 0;
for(int i = 0; i < 100; i++) {
    result += i;
}
```
Аналогичный код на Python будет выглядеть следующим образом
```python
# Python code
result = 0
for i in range(100):
    result += i
```
Как видно из этого кода при объявлении переменной в C указывается тип этой переменной, а в Python этот тип выводится динамически во время выполнения. Язык C относится к языкам со статической типизацией, а Python с динамической типизацией. В первом случае можно добиться высокой скорости исполнения, но ценой более длительного времени затрачиваемого на разработку. Во втором случае время программиста, затрачиваемое на написание кода сокращается, но ценой значительно долгого времени выполнения программы.

### Тип Integer в Python это целая структура
Стандартная реализация Python также написана на C и пример кода на Python, приведенного выше, фактически является оберткой на языке C. Когда объявляется переменная с целочисленным значением Python использует следующую структуру C:
```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```
Т.е. тип `Integer` в Python это не просто целочисленное значение, а целая структура, которая содержит следующую информацию:
- ``ob_refcnt``, количество ссылок на объект для работы сборщика мусора
- ``ob_type``, тип переменной
- ``ob_size``, размер объекта
- ``ob_digit``, фактическое значение

Сама переменная `result` в вышеприведенном примере это всего лишь ссылка (т.е. адрес в памяти) на объект.
### Список в Python это список структур
Давайте рассмотрим следующий код

In [4]:
l = [True, "42", 3.0, 4]
[type(item) for item in l]

[bool, str, float, int]

Данный код создает список в Python с разными значениями и выводит тип каждого из этих значений. Как видно из примера в списке представлены значения четырех типов. Такое возможно благодаря динамической типизации Python, что часто бывает очень удобным. Однако такая гибкость имеет свою цену: для каждого элемента списка нужно хранить отдельную структуру со всей необходимой информацией. В итоге список хранит лишь ссылки на адреса в памяти, где хранятся фактические значения списка. Если бы в списке можно было хранить только один тип данных, то большая часть этой информации была бы избыточной. Ниже схематически представлена разница в хранении данных в NumPy и Python.

![Array Memory Layout](img/array_vs_list.png)

Стоит отметить, что в Python начиная с версии 3.3 представлен модуль `array`, который дает возможность создания массивов с фиксированной длиной и одним типом. Однако, массивы NumPy значительно полезнее, так как кроме эффективного хранения данных также поддерживаются разные операции над ними.

### Создание массивов NumPy
Массивы NumPy можно создать несколькими способами. Например, из списков Python

In [5]:
np.array([1, 2, 3, 4, 5])

array([1, 2, 3, 4, 5])

При создании можно указать тип массива

In [6]:
np.array([1, 2, 3, 4], dtype='float32')

array([ 1.,  2.,  3.,  4.], dtype=float32)

Или создать двумерный массив

In [7]:
np.array([range(i, i + 4) for i in [1, 2, 3]])

array([[1, 2, 3, 4],
       [2, 3, 4, 5],
       [3, 4, 5, 6]])

Можно создать массивы с нулями или единицами с заданным размером массива

In [8]:
print(np.zeros(10, dtype=int))
print(np.ones((5, 3), dtype=float))

[0 0 0 0 0 0 0 0 0 0]
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]


Или массив, заполненный любым другим значением

In [9]:
np.full((3, 7), 42)

array([[42, 42, 42, 42, 42, 42, 42],
       [42, 42, 42, 42, 42, 42, 42],
       [42, 42, 42, 42, 42, 42, 42]])

Диапазон значений с заданным шагом получается следующим образом

In [10]:
np.arange(10, 20, 2)

array([10, 12, 14, 16, 18])

Массив из n чисел равномерно распределенных в заданном диапазоне можно получить с помощью функции `np.linspace`

In [11]:
np.linspace(0, 1, 5)

array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ])

Можно также создать двумерный массив, у которого по диагонали единицы, а во всех остальных местах нули. Такой массив имеет специальное значение в математике и называется единичной матрицей (англ. identity matrix, отсюда сокращенное название eye)

In [12]:
np.eye(5)

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

### Генерация случайных чисел
Можно создавать массивы из генератора случайных чисел, что имеет много важных приложений. Например, можно генерировать равномерно распределенные случайные величины между 0 и 1

In [13]:
np.random.random((3, 3))

array([[ 0.6820219 ,  0.39131121,  0.90829172],
       [ 0.21743442,  0.66452465,  0.71960584],
       [ 0.04015015,  0.2936675 ,  0.80376751]])

Или случайных целых чисел

In [14]:
np.random.randint(0, 10, (3, 3))

array([[4, 2, 0],
       [6, 8, 1],
       [1, 5, 9]])

Генерация случайных чисел с нормальным распределением со средним значением 0 и стандартным отклонением 1

In [15]:
np.random.normal(0, 1, (3, 3))

array([[-0.73019538,  1.93524447,  1.19911828],
       [-0.91108715, -0.55575574, -1.01890768],
       [-0.68275845, -1.08175951, -0.65483112]])

### Атрибуты массивов
У каждого массива есть набор полезных атрибутов, из которых можно узнать информацию о массиве. Начнем с создания массива

In [16]:
import numpy as np
np.random.seed(0)

a3 = np.random.randint(10, size=(3, 4, 5))  # Three-dimensional array
a3

array([[[5, 0, 3, 3, 7],
        [9, 3, 5, 2, 4],
        [7, 6, 8, 8, 1],
        [6, 7, 7, 8, 1]],

       [[5, 9, 8, 9, 4],
        [3, 0, 3, 5, 0],
        [2, 3, 8, 1, 3],
        [3, 3, 7, 0, 1]],

       [[9, 9, 0, 4, 7],
        [3, 2, 7, 2, 0],
        [0, 4, 5, 5, 6],
        [8, 4, 1, 4, 9]]])

У каждого массива есть атрибуты `ndim` (количество измерений), `shape` (размер каждого измерения) и `size` (количество всех элементов массива)

In [17]:
print('ndim:', a3.ndim)
print('shape:', a3.shape)
print('size:', a3.size)

ndim: 3
shape: (3, 4, 5)
size: 60


Атрибут `dtype` показывает тип данных в массиве

In [18]:
a3.dtype

dtype('int32')

Атрибут `itemsize` показывает размер каждого элемента массива в байтах, а атрибут `nbytes` показывает общий размер всего массива в байтах

In [19]:
print('itemsize: ', a3.itemsize)
print('nbytes: ', a3.nbytes)

itemsize:  4
nbytes:  240


### Индексация массива
NumPy поддерживает индексацию в стиле списков Python

In [20]:
a1 = np.random.randint(10, size=10)
a1

array([8, 1, 1, 7, 9, 9, 3, 6, 7, 2])

In [21]:
a1[5]

9

In [22]:
a1[-2]

7

Аналогично для двухмерных массивов

In [23]:
a2 = np.random.randint(10, size=(3, 5))
a2

array([[0, 3, 5, 9, 4],
       [4, 6, 4, 4, 3],
       [4, 4, 8, 4, 3]])

In [24]:
a2[0, 0]

0

In [25]:
a2[2, -1]

3

Можно присвоить значение элементу массива

In [26]:
a2[0, 0] = 42
a2

array([[42,  3,  5,  9,  4],
       [ 4,  6,  4,  4,  3],
       [ 4,  4,  8,  4,  3]])

### Срезы и подмассивы
NumPy следует синтаксису Python для получения подмассивов:
```Python
a[start:stop:step]
```
Если какой-то из этих параметров не указаны, то по умолчанию используется `start=0`, `stop=size of dimension` и `step=1`.

### Примеры для одномерных массивов

In [27]:
a1[:5]

array([8, 1, 1, 7, 9])

In [28]:
a1[3:7]

array([7, 9, 9, 3])

In [29]:
a1[4:]

array([9, 9, 3, 6, 7, 2])

In [30]:
a1[::2]

array([8, 1, 9, 3, 7])

In [31]:
a1[::-1]

array([2, 7, 6, 3, 9, 9, 7, 1, 1, 8])

In [32]:
a1[2:-1:3]

array([1, 9, 7])

### Примеры для многомерных массивов
Для многомерных массивов срезы (slice) указываются для каждого измерения по отдельности через запятую

In [33]:
a2

array([[42,  3,  5,  9,  4],
       [ 4,  6,  4,  4,  3],
       [ 4,  4,  8,  4,  3]])

In [34]:
a2[:2, :4] # two rows, four columns

array([[42,  3,  5,  9],
       [ 4,  6,  4,  4]])

In [35]:
a2[:2, ::-1] # two rows, columns in reverse order

array([[ 4,  9,  5,  3, 42],
       [ 3,  4,  4,  6,  4]])

### Выбор строк и колонок
Для выбора одной строки или колонки многомерного массива целиком можно использовать пустой срез (:)

In [36]:
a2[:, 1] # choose all rows for second column

array([3, 6, 4])

In [37]:
a2[2, :] # choose all columns for third row

array([4, 4, 8, 4, 3])

### Срезы не являются копиями
Срезы массивов не создаются копированием соответствующих значений. Если изменить значение подмассива, то соответствующее значение в оригинальном массиве тоже изменится

In [39]:
a2

array([[42,  3,  5,  9,  4],
       [ 4,  6,  4,  4,  3],
       [ 4,  4,  8,  4,  3]])

In [38]:
a2_sub = a2[:2, :2]
print(a2_sub)

[[42  3]
 [ 4  6]]


In [40]:
a2_sub[1, 1] = 13
a2

array([[42,  3,  5,  9,  4],
       [ 4, 13,  4,  4,  3],
       [ 4,  4,  8,  4,  3]])

Для создания срезов с копированием можно использовать метод `copy()` массива

In [41]:
a2_sub = a2[:2, :2].copy()
a2_sub[1,1] = 321
print(a2)

[[42  3  5  9  4]
 [ 4 13  4  4  3]
 [ 4  4  8  4  3]]


### Изменение формы массива (reshaping)
Иногда необходимо изменить форму массива. Например, разложить одномерный массив в несколько строк

In [44]:
x = np.arange(1, 17).reshape((4, 4))
print(x)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


В некоторых случая необходимо создать из одномерного массива двумерный массив с одной строкой или колонкой

In [45]:
x = np.array([1, 2, 3])
print(x.reshape((1, 3)))

[[1 2 3]]
[[1]
 [2]
 [3]]


In [46]:
print(x.reshape((3, 1)))

[[1]
 [2]
 [3]]


Такого же результата можно добиться с помощью объекта `np.newaxis`

In [48]:
x[np.newaxis, :]

array([[1, 2, 3]])

In [50]:
x[:, np.newaxis]

array([[1],
       [2],
       [3]])

### Объединение массивов

Можно объеденить несколько массивов в один с помощью функции `np.concatenate`

In [52]:
x = np.array([1, 2, 3])
y = np.array([3, 2, 1])
np.concatenate([x, y])

array([1, 2, 3, 3, 2, 1])

Можно объеденить многомерные массивы

In [53]:
x = np.array([[1, 2, 3],
              [4, 5, 6]])
y = np.array([[7, 8, 9],
              [10, 11, 12]])
np.concatenate([x, y])

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

Также можно указать измерение, по которому нужно объеденить, с помощью параметра `axis`

In [54]:
np.concatenate([x, y], axis=1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

Есть также методы `np.vstack` и `np.hstack` для объединения массивов в строки или столбцы. Эти методы более интуитывно понятны, особенно когда размер одного измерения массивов не одинаковый

In [55]:
x = np.array([1, 2, 3])
y = np.array([[9, 8, 7],
              [6, 5, 4]])

# vertically stack the arrays
np.vstack([x, y])

array([[1, 2, 3],
       [9, 8, 7],
       [6, 5, 4]])

In [56]:
x = np.array([[1], 
              [2], 
              [3]])
y = np.array([[9, 8, 7],
              [6, 5, 4],
              [3, 2, 1]])

# vertically stack the arrays
np.hstack([x, y])

array([[1, 9, 8, 7],
       [2, 6, 5, 4],
       [3, 3, 2, 1]])